In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver

In [2]:
from langgraph.graph.message import add_messages
# add_messages is redured to avoid circular import issues
class ChatState(TypedDict):

    messages: Annotated[list[BaseMessage], add_messages]

In [3]:
llm = ChatOpenAI()


def chat_node(state: ChatState):

    # take user query from state
    messages = state['messages']

    # send to llm
    response = llm.invoke(messages)

    # response store state
    return {'messages': [response]}

In [5]:
checkpointer = MemorySaver()
graph = StateGraph(ChatState)

# add nodes
graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile(checkpointer=checkpointer)

In [6]:
initial_state = {
    'messages': [HumanMessage(content='What is the capital of india')]
}

# chatbot.invoke(initial_state)['messages'][-1].content
chatbot.invoke(initial_state)['messages']

ValueError: Checkpointer requires one or more of the following 'configurable' keys: thread_id, checkpoint_ns, checkpoint_id

In [ ]:
#multiple user interactions with chatbot with unique thread id
thread_id='1'
while True:
    user_message = input("Type here: ")
    print("User:" ,user_message)
    if user_message.strip().lower() in ['exit', 'quit',"bye","thank you",'thanks','goodbye','see you','stop','end','nothing','exit chat','exit conversation','terminate','close chat','close conversation']:
        break

    config={'configurable': {'thread_id' : thread_id}}
    response = chatbot.invoke({'messages': [HumanMessage(content=user_message)]},config=config)

    # response = chatbot.invoke(initial_state)['messages'][-1].content
    print("AI:", response['messages'][-1].content)

User: hi my name is amarendra
AI: Nice to meet you, Amarendra! How can I assist you today?
User: what is my name
AI: Your name is Amarendra.
User: add 10 and 100 number
AI: 10 + 100 equals 110.
User: now add to 120
AI: If we add 110 (the result of 10 + 100) to 120, the total is 230.
User: and multiply into 2
AI: If we multiply 230 by 2, the result is 460.
User: and add my sirname kumar
AI: If we add your surname "Kumar" to 460, it would just be Amarendra Kumar.
User: add my sir name in my name
AI: Got it! Your full name is Amarendra Kumar.
User: and my optinal name is bahubali
AI: That's a powerful and regal optional name! So, your full name with your optional name is Amarendra Kumar Bahubali.
User: no my  2nd name is bahubali
AI: Understood! Your full name is Amarendra Bahubali Kumar.
User: so what i my name
AI: Your full name is Amarendra Bahubali Kumar.
User: no bahubali is my office name
AI: Got it! Your full name is Amarendra Kumar and your office name is Bahubali.
User: yes
AI: G